<h2 align='center'>PROJECT 4 - ACCRE</h2>

#### Using just the fullsample.csv jobs dataset:

### 1. Calculate some descriptive statistics for how many jobs per hour are being completed. What does the completions per hour look like over the time span of the dataset? Are there weekly trends, and has it been increasing over the last year?

In [1]:
from dateutil.parser import parse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
jobs = pd.read_csv('../data/fullsample.csv')

In [ ]:
jobs.info()
display(jobs.head())

In [3]:
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'],
                               errors = 'coerce')

In [4]:
jobs['END'] = pd.to_datetime(jobs['END'],
                             errors = 'coerce')

In [5]:
jobs = jobs.sort_values('END')

In [6]:
jobs = jobs[jobs['END'].notnull()]

In [7]:
jobs = jobs[jobs['BEGIN'].notnull()]

In [8]:
jobs['JOBLENGTH'] = jobs['END'] - jobs['BEGIN']

In [9]:
jobs['REQMEMTOT'] = jobs['REQMEM'] * jobs['CPUS']

In [ ]:
# jobs['REQMEMPERCORE'] = jobs['REQMEMTOT'] / jobs['CPUS']

In [10]:
# TO DO 11/2 - LIST 'COMPLETED' JOBS, NOT ALL JOBS

jobs['ROLL1H'] =  jobs.rolling('1h', 
                               on = 'END')['JOBID']\
                      .count()

In [ ]:
# GROUP BY DAY?

# GROUPBYROLL1D = 
# jobs.groupby(['END'])\
#     .rolling('1d', 
#          on = 'END')['JOBID']\
#     .count()\
#     .reset_index()\
#     .set_index('END')
#            
# GROUPBYROLL1D

roll1day = jobs.rolling('1d', 
                        on = 'END')['JOBID']\
               .count()

print(roll1day.describe())

roll1week.plot(kind = 'line',
               figsize=(12,6));

In [ ]:
roll1week = jobs.rolling('7d', 
                         on = 'END')['JOBID']\
                .count()

print(roll1week.describe())
#roll1week.plot(kind = 'line',
#               figsize=(12,6));

In [ ]:
roll1month = jobs.rolling('30d', 
                          on = 'END')['JOBID']\
                 .count()

print(roll1month.describe())

In [ ]:
# jobs.plot(kind = 'line',
#           x = 'END',
#           y = 'ROLL1H');

In [ ]:
# jobs.groupby(pd.Grouper(key = 'JOBLENGTH',
 #                       freq = '1W')).mean()

In [ ]:
jobs['JOBLENGTH'].describe()

In [ ]:
jobs['ROLL1H'].describe()

In [ ]:
jobs.plot(kind = 'line',
          x = 'END',
          y = 'JOBLENGTH',
          figsize=(12,6));

In [ ]:
jobs.groupby(pd.Grouper(key = 'JOBLENGTH',
                        freq = '1W'))\
    .plot(kind = 'line',
          x = 'END',
          y = 'JOBLENGTH');

In [ ]:
# THIS TAKES WAY TOO LONG TO GRAPH - TODO 10/30
# 
# jobs.plot(kind = 'line',
#           x = 'END',
#           y = 'ROLL1H');

In [ ]:
# jobs['END'].dt.date

In [ ]:
# IS describe() USEFUL HERE? -- TODO 10/30

# jobs.groupby('END'.dt.date)['ROLL1H']\
#     .describe()

In [ ]:
# 'GROUPBYJOBSTATE' = 
jobs\
    .groupby('STATE')['JOBID']\
    .count()\
    .sort_values(ascending=False)
#   .describe()
# GROUPBYJOBSTATE

In [ ]:
# jobs['STATE'].unique()

In [ ]:
# zero_exits = jobs[(jobs['STATE']!='COMPLETED') & (jobs['EXITCODE'] == '0:0')]
# print(zero_exits['STATE'].unique())

#### Using just the fullsample.csv jobs dataset:

### 2. Does the job state affect completions per hour? i.e. if I only look at jobs with exit code 0:0 in the "COMPLETED" state, is that a similar number of completions per hour as with all jobs, failed or cancelled? This will indicate if the load on the scheduler is by user design or is a result of users not sufficiently testing their jobs before submitting very large arrays. We also expect that most job completions will be in the "production" partition, but is this actually true?

In [ ]:
'''
EXITCODE 0:0            = SUCCESS
         0:15, 0:9, 0:2 = CANCELLED
         1:0, 2.0       = FAILED
'''
# GROUPBYEXITCOUNTS = 
#                    jobs\
#                       .groupby('EXITCODE')['JOBID']\
#                       .count()\
#                       .sort_values(ascending=False)
# GROUPBYEXITCOUNTS

In [ ]:
# MOVING COUNT GROUPING BY EXIT CODE -- 
# 
# group_by_exit_code = 
#                     (jobs    # jobs[jobs['END'].notnull()]
#                          .groupby('EXITCODE')
#                          .rolling('1h', 
#                                   on = 'END')['JOBID']
#                          .count()
#                          .reset_index()
#                          .set_index('END')
#                     )
# group_by_exit_code

### Next, use the two log files to determine time intervals (hours) when the scheduler was unresponsive. Do this by looking for records that are the "sbatch" command from user 9204 that have return code 1 and an execution time of ~20 seconds (at least more than 15). These are commands where the scheduler timed out in responding.

### 3.	Calculate some descriptive statistics about how often the scheduler was unresponsive, how long these periods of time were, and create a time series plot of when the scheduler was having difficulties.

In [11]:
ce5_raw = pd.read_table("../data/slurm_wrapper_ce5.log", 
                        engine='python',
                        sep=" - ", 
                        names=['DATE', 
                               'USER', 
                               'RETRY',
                               'TIMELAPS',
                               'RETURNCODE',
                               'COMMAND']
                       ) 

In [ ]:
ce5_raw.info()
display(ce5_raw.head())

In [13]:
ce5_raw['DATE'] = pd.to_datetime(ce5_raw.DATE)

In [12]:
ce5_raw['SERVERID'] = 'ce5'

In [ ]:
# ce5_hrs_range = ce5_raw['DATE'].max() - ce5_raw['DATE'].min()
# display(ce5_hrs_range)

In [14]:
ce6_raw = pd.read_table("../data/slurm_wrapper_ce6.log", 
                        engine='python',
                        sep=" - ", 
                        names=['DATE', 
                               'USER', 
                               'RETRY',
                               'TIMELAPS',
                               'RETURNCODE',
                               'COMMAND']
                       ) 

In [ ]:
ce6_raw.info()
display(ce6_raw.head())

In [16]:
ce6_raw['DATE'] = pd.to_datetime(ce6_raw.DATE)

In [15]:
ce6_raw['SERVERID'] = 'ce6'

In [ ]:
# ce6_hrs_range = ce6_raw['ce6_DateTime'].max() - ce6_raw['ce6_DateTime'].min()
# display(ce6_hrs_range)

In [17]:
logs = ce5_raw.copy()

In [18]:
# APPEND TWO SERVERS' LOGS INTO A SINGLE DATA FRAME

logs.append(ce6_raw)

,DATE,USER,RETRY,TIMELAPS,RETURNCODE,COMMAND,SERVERID
0,2020-10-16 08:15:39.278699,user 0,retry 0,time 0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '...",ce5
1,2020-10-16 08:18:08.313309,user 0,retry 0,time 0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '...",ce5
2,2020-10-16 08:22:48.128689,user 0,retry 0,time 0.07547116279602051,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '...",ce5
3,2020-10-16 08:25:13.257408,user 0,retry 0,time 0.09484362602233887,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '...",ce5
4,2020-10-16 08:31:01.460723,user 0,retry 0,time 0.07498788833618164,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '...",ce5
...,...,...,...,...,...,...,...
4776515,2021-10-07 21:59:35.014602,user 9221,retry 0,time 0.060086965560913086,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u...",ce6
4776516,2021-10-07 21:59:35.238970,user 9202,retry 0,time 0.09804415702819824,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u...",ce6
4776517,2021-10-07 21:59:57.265189,user 9203,retry 0,time 0.02454972267150879,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u...",ce6
4776518,2021-10-07 22:00:04.024360,user 9201,retry 0,time 0.03941917419433594,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u...",ce6


In [19]:
# TO DO - USE row num AS 'JOBID'?

logs['JOBID'] = '0'

In [ ]:
# TO DO - SPLIT TIMELAPS COLUMN AND TURN INTO TIMELAPSE OR OTHER MATH DATA TYPE

In [27]:
# TO SEE ENTIRE 'COMMAND' COLUMN - pd.set_option('max_colwidth', None), THEN
# pd.reset_option('max_colwidth')

In [25]:
logs['COMMAND'].head()

0    command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-10']
1    command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-10']
2    command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-10']
3    command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-10']
4    command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-14']
Name: COMMAND, dtype: object

In [22]:
logs[(logs['USER']=='user 9204')\
      & (logs['RETURNCODE']=='returncode 1')
#     & (logs['TIMELAPS']> ...) # WHAT IS UNIT HERE?  + MUST CONVERT DATA TYPE
#     & (logs['COMMAND'] ... )  # NEED TO REGEX PARSE sbatch, etc FROM 'COMMAND'     
    ]

,DATE,USER,RETRY,TIMELAPS,RETURNCODE,COMMAND,SERVERID,JOBID
564,2020-10-16 08:59:38.926896,user 9204,retry 0,time 0.1231698989868164,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '...",ce5,0
565,2020-10-16 08:59:38.986336,user 9204,retry 1,time 0.05905652046203613,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '...",ce5,0
566,2020-10-16 08:59:39.133897,user 9204,retry 2,time 0.1472628116607666,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '...",ce5,0
683,2020-10-16 09:11:17.798315,user 9204,retry 0,time 0.02103400230407715,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '...",ce5,0
684,2020-10-16 09:11:17.817819,user 9204,retry 1,time 0.019124746322631836,returncode 1,"command ['/usr/bin/scontrol', 'show', 'job', '...",ce5,0
...,...,...,...,...,...,...,...,...
4729891,2021-10-02 16:01:42.684681,user 9204,retry 0,time 21.28179407119751,returncode 1,"command ['/usr/bin/squeue', '-o', '%i %T', '-u...",ce5,0
4730187,2021-10-02 16:34:51.632533,user 9204,retry 0,time 20.060213804244995,returncode 1,"command ['/usr/bin/squeue', '-o', '%i %T', '-u...",ce5,0
4730701,2021-10-02 17:33:19.126956,user 9204,retry 0,time 20.50884461402893,returncode 1,"command ['/usr/bin/squeue', '-o', '%i %T', '-j...",ce5,0
4730705,2021-10-02 17:33:39.156362,user 9204,retry 1,time 20.028889894485474,returncode 1,"command ['/usr/bin/squeue', '-o', '%i %T', '-j...",ce5,0


### 4.	Finally, combine the time series information from the two datasets together to see how well correlated heavy job-completion load is with the unresponsiveness, and to see if there is some threshold of job completions per hour that generally results in unresponsiveness.

In [ ]:
# APPENDING BOTH SERVERS' LOGS INTO ONE logs DATAFRAME   - TODO 11/2

# logs = pd.append(left = ce5_raw, right = ce6_raw,
#                 )